### Decision Tree, Random Forest and Gradient Boosting Classifier Project

In this project we will code along with some data and test out 3 different tree methods:

* A single decision tree
* A random forest
* A gradient boosted tree classifier
    
We will be using a college dataset to try to classify colleges as Private or Public based off these features:

    Private A factor with levels No and Yes indicating private or public university
    Apps Number of applications received
    Accept Number of applications accepted
    Enroll Number of new students enrolled
    Top10perc Pct. new students from top 10% of H.S. class
    Top25perc Pct. new students from top 25% of H.S. class
    F.Undergrad Number of fulltime undergraduates
    P.Undergrad Number of parttime undergraduates
    Outstate Out-of-state tuition
    Room.Board Room and board costs
    Books Estimated book costs
    Personal Estimated personal spending
    PhD Pct. of faculty with Ph.D.’s
    Terminal Pct. of faculty with terminal degree
    S.F.Ratio Student/faculty ratio
    perc.alumni Pct. alumni who donate
    Expend Instructional expenditure per student
    Grad.Rate Graduation rate

In [1]:
from pyspark.sql import SparkSession
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier,RandomForestClassifier,GBTClassifier

In [2]:
spark = SparkSession.builder.appName('Trees').getOrCreate()

In [3]:
df = spark.read.csv('../datasets/College_Data',inferSchema=True,header=True)

In [4]:
df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F.Undergrad: integer (nullable = true)
 |-- P.Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room.Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S.F.Ratio: double (nullable = true)
 |-- perc.alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad.Rate: integer (nullable = true)



### Data Pre-Processing

In [5]:
from pyspark.ml.feature import StringIndexer

In [6]:
indexer = StringIndexer(inputCol='Private',outputCol='PrivateIndex')

In [7]:
output = indexer.fit(df).transform(df)

In [16]:
# Renaming columns so that we can pass it in assembler.
output = output.withColumnRenamed('F.Undergrad','F_Undergrad').withColumnRenamed('P.Undergrad','P_Undergrad')                             .withColumnRenamed('S.F.Ratio','S_F_Ratio').withColumnRenamed('perc.alumni','perc_alumni')                                 .withColumnRenamed('Grad.Rate','Grad_Rate').withColumnRenamed('Room.Board','Room_Board')

In [17]:
output.columns

['_c0',
 'Private',
 'Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate',
 'PrivateIndex']

In [19]:
assembler = VectorAssembler(inputCols=['Apps',
 'Accept',
 'Enroll',
 'Top10perc',
 'Top25perc',
 'F_Undergrad',
 'P_Undergrad',
 'Outstate',
 'Room_Board',
 'Books',
 'Personal',
 'PhD',
 'Terminal',
 'S_F_Ratio',
 'perc_alumni',
 'Expend',
 'Grad_Rate'],outputCol='features')

In [20]:
assembled_data = assembler.transform(output)

In [21]:
assembled_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Private: string (nullable = true)
 |-- Apps: integer (nullable = true)
 |-- Accept: integer (nullable = true)
 |-- Enroll: integer (nullable = true)
 |-- Top10perc: integer (nullable = true)
 |-- Top25perc: integer (nullable = true)
 |-- F_Undergrad: integer (nullable = true)
 |-- P_Undergrad: integer (nullable = true)
 |-- Outstate: integer (nullable = true)
 |-- Room_Board: integer (nullable = true)
 |-- Books: integer (nullable = true)
 |-- Personal: integer (nullable = true)
 |-- PhD: integer (nullable = true)
 |-- Terminal: integer (nullable = true)
 |-- S_F_Ratio: double (nullable = true)
 |-- perc_alumni: integer (nullable = true)
 |-- Expend: integer (nullable = true)
 |-- Grad_Rate: integer (nullable = true)
 |-- PrivateIndex: double (nullable = false)
 |-- features: vector (nullable = true)



In [22]:
final_data = assembled_data.select('features','PrivateIndex')

In [23]:
final_data.show()

+--------------------+------------+
|            features|PrivateIndex|
+--------------------+------------+
|[1660.0,1232.0,72...|         0.0|
|[2186.0,1924.0,51...|         0.0|
|[1428.0,1097.0,33...|         0.0|
|[417.0,349.0,137....|         0.0|
|[193.0,146.0,55.0...|         0.0|
|[587.0,479.0,158....|         0.0|
|[353.0,340.0,103....|         0.0|
|[1899.0,1720.0,48...|         0.0|
|[1038.0,839.0,227...|         0.0|
|[582.0,498.0,172....|         0.0|
|[1732.0,1425.0,47...|         0.0|
|[2652.0,1900.0,48...|         0.0|
|[1179.0,780.0,290...|         0.0|
|[1267.0,1080.0,38...|         0.0|
|[494.0,313.0,157....|         0.0|
|[1420.0,1093.0,22...|         0.0|
|[4302.0,992.0,418...|         0.0|
|[1216.0,908.0,423...|         0.0|
|[1130.0,704.0,322...|         0.0|
|[3540.0,2001.0,10...|         1.0|
+--------------------+------------+
only showing top 20 rows



### Train Test Split


In [24]:
train_data, test_data = final_data.randomSplit([0.7,0.3])

In [38]:
dtc = DecisionTreeClassifier(labelCol='PrivateIndex',featuresCol='features')
rfc = RandomForestClassifier(labelCol='PrivateIndex',featuresCol='features',numTrees=100)
gbc = GBTClassifier(labelCol='PrivateIndex',featuresCol='features',maxIter=50)

In [39]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbc_model = gbc.fit(train_data)

In [40]:
dtc_pred = dtc_model.transform(test_data)
rfc_pred = rfc_model.transform(test_data)
gbc_pred = gbc_model.transform(test_data)

### Evaluation

In [28]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

binary_eval = BinaryClassificationEvaluator(labelCol='PrivateIndex')

In [29]:
print("Decision Tree:")
print(binary_eval.evaluate(dtc_pred))

Decision Tree:
0.8906419431793537


In [30]:
print("Random Forest:")
print(binary_eval.evaluate(rfc_pred))

Random Forest:
0.9775536759921929


In [33]:
print("Gradient Boosting Tree:")
print(binary_eval.evaluate(gbc_pred))

Gradient Boosting Tree:
0.9617219692040772


In [41]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [42]:
acc_eval = MulticlassClassificationEvaluator(labelCol='PrivateIndex',metricName='accuracy')

In [43]:
rfc_acc = acc_eval.evaluate(rfc_pred)
dtc_acc = acc_eval.evaluate(dtc_pred)
gbc_acc = acc_eval.evaluate(gbc_pred)

In [44]:
print("Here are the results!")
print('-'*80)
print('A single decision tree had an accuracy of: {0:2.2f}%'.format(dtc_acc*100))
print('-'*80)
print('A random forest ensemble had an accuracy of: {0:2.2f}%'.format(rfc_acc*100))
print('-'*80)
print('A ensemble using GBT had an accuracy of: {0:2.2f}%'.format(gbc_acc*100))

Here are the results!
--------------------------------------------------------------------------------
A single decision tree had an accuracy of: 91.71%
--------------------------------------------------------------------------------
A random forest ensemble had an accuracy of: 94.01%
--------------------------------------------------------------------------------
A ensemble using GBT had an accuracy of: 90.32%


`We can tune hyperparameters of these ML models to get more accuracy`